In [4]:
import autograd.numpy as np

import pymanopt
from pymanopt.manifolds import Elliptope
from pymanopt.solvers import ConjugateGradient

SUPPORTED_BACKENDS = ("Autograd")


def create_cost(manifold, epsilon, backend):
    @pymanopt.function.Autograd(manifold)
    def cost(X):
        Y = X @ X.T
        # Shift the exponentials by the maximum value to reduce numerical
        # trouble due to possible overflows.
        s = np.triu(Y, 1).max()
        expY = np.exp((Y - s) / epsilon)
        # Zero out the diagonal
        expY -= np.diag(np.diag(expY))
        u = np.triu(expY, 1).sum()
        return s + epsilon * np.log(u)

    return cost


backend=SUPPORTED_BACKENDS[0]
quiet=False
dimension = 3  # Dimension of the embedding space, i.e. R^k
num_points = 24  # Points on the sphere
# This value should be as close to 0 as affordable. If it is too close to
# zero, optimization first becomes much slower, than simply doesn't work
# anymore because of floating point overflow errors (NaN's and Inf's start
# to appear). If it is too large, then log-sum-exp is a poor approximation
# of the max function, and the spread will be less uniform. An okay value
# seems to be 0.01 or 0.001 for example. Note that a better strategy than
# using a small epsilon straightaway is to reduce epsilon bit by bit and to
# warm-start subsequent optimization in that way. Trustregions will be more
# appropriate for these fine tunings.
epsilon = 0.005

manifold = Elliptope(num_points, dimension)
cost = create_cost(manifold, epsilon, backend)
problem = pymanopt.Problem(manifold, cost)
if quiet:
    problem.verbosity = 0

solver = ConjugateGradient(mingradnorm=1e-8, maxiter=1e5)
Yopt = solver.solve(problem)

Xopt = Yopt @ Yopt.T
maxdot = np.triu(Xopt, 1).max()
print("Maximum angle between any two points:", maxdot)

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
     0       +1.0247437994518909e+00    0.00000000e+00    
Terminated - min grad norm reached after 1 iterations, 0.00 seconds.

Maximum angle between any two points: 1.0


In [1]:
import autograd.numpy as anp
import pymanopt
import pymanopt.manifolds
import pymanopt.solvers

anp.random.seed(42)

dim = 3
manifold = pymanopt.manifolds.Sphere(dim)

matrix = anp.random.normal(size=(dim, dim))
matrix = 0.5 * (matrix + matrix.T)

@pymanopt.function.Autograd(manifold)
def cost(point):
    return -point @ matrix @ point

problem = pymanopt.Problem(manifold=manifold, cost=cost)

solver = pymanopt.solvers.SteepestDescent()
solution = solver.solve(problem)

eigenvalues, eigenvectors = anp.linalg.eig(matrix)
dominant_eigenvector = eigenvectors[:, eigenvalues.argmax()]

print("Dominant eigenvector:", dominant_eigenvector)
print("Pymanopt solution:", solution)

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +1.1041943339110254e+00    5.65626470e-01    
   2         +5.2849633289004561e-01    8.90742722e-01    
   3         -8.0741058657312559e-01    2.23937710e+00    
   4         -1.2667369971251594e+00    1.59671326e+00    
   5         -1.4100298597091836e+00    1.11228845e+00    
   6         -1.5219408277812505e+00    2.45507203e-01    
   7         -1.5269956262562046e+00    6.81712914e-02    
   8         -1.5273114803528709e+00    3.40941735e-02    
   9         -1.5273905588875487e+00    1.70222768e-02    
  10         -1.5274100956128560e+00    8.61140952e-03    
  11         -1.5274154319869837e+00    3.90706914e-03    
  12         -1.5274156215853507e+00    3.62943721e-03    
  13         -1.5274162595152783e+00    2.47643452e-03    
  14         -1.5274168030609154e+00    3.66398414e-04    
  15         -1.5274168133149475e+00    1.